### Star Ratings Results
- Input from AimBig notebook, stat_regression 
- 6 steps of the star rating results
- GroupBy object = (Contract ID, ESA Code, Speciality_site_type_code)

In [1]:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# reading in file with numerator/denominator for each participant, performance measure against contract
df_merged = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_final.csv')
# creating groupBy object 
df_merged = df_merged.drop_duplicates()
#df_group = df_merged.groupby(['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])

df_merged.columns

df_merged = df_merged.drop(columns=['Unnamed: 0'])

Columns missing due to no outcomes = 'Num_26_path', 'Num_26_wrkast', 'Den_26_path', 'Den_26_wrkast', 'Den_52_path', 'Num_52_path'
- set as 0 

In [2]:
df_merged[['Num_26_path', 'Num_26_wrkast', 'Den_26_path','Den_26_wrkast', 'Den_52_path', 'Num_52_path']] = 0

In [3]:
print(df_merged.isna().sum())

Program                         0
JOB_SEEKER_ID                   0
CONTRACT_ID                     0
SITE_CODE                       0
SITE_DESCRIPTION                0
ESA_CODE                        0
ESA Name                        0
State                           0
SPECIALIST_SITE_TYPE_CODE       0
Num_13_full                  3813
Den_13_full                  3813
Num_13_exp                   3813
Num_26_full                  4768
Den_26_full                  4768
Num_26_full_exp              4768
Num_26_path                     0
Den_26_path                     0
Num_26_path_exp              5312
Num_26_wrkast                   0
Den_26_wrkast                   0
Num_26_wrkast_exp            5312
Num_52_full                  5193
Den_52_full                  5193
Num_52_full_exp              5188
Num_52_path                     0
Den_52_path                     0
Num_52_path_exp              5188
dtype: int64


### Step 1) Calculating Actual Performance
#### Actual Performance
groupBy key = (Contract ID, ESA Code, Speciality_site_type_code)

##### Pre-Quarter

- Count NaNs -> place as 0

In [4]:
df_merged = df_merged.fillna(value = 0)
df_merged

,Program,JOB_SEEKER_ID,CONTRACT_ID,SITE_CODE,SITE_DESCRIPTION,ESA_CODE,ESA Name,State,SPECIALIST_SITE_TYPE_CODE,Num_13_full,...,Num_26_path_exp,Num_26_wrkast,Den_26_wrkast,Num_26_wrkast_exp,Num_52_full,Den_52_full,Num_52_full_exp,Num_52_path,Den_52_path,Num_52_path_exp
0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,0.0,...,0.0,0,0,0.0,1.0,1.0,0.0,0,0,0.0
1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0
2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,1.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0
3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0
4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5307,DMS,2730016019,0212841E,Y947,AimBig Employment MERRYLANDS,4CWS,Northern Sydney,NSW,AALL,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0
5308,DMS,2696040,0212857B,ML77,AimBig Employment WOLLONGONG,4WOL,South East Sydney,NSW,MUSK,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0
5309,DMS,5892574007,0212825E,MM77,AimBig Employment MELBOURNE,4YAR,Northern Victoria,VIC,MUSK,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0
5310,DMS,832617003,0212823C,YA97,AimBig Employment Pty Ltd ST ALBANS,4WES,Western Victoria,VIC,MUSK,0.0,...,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0.0


In [5]:
df_merged.columns

Index(['Program', 'JOB_SEEKER_ID', 'CONTRACT_ID', 'SITE_CODE',
       'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State',
       'SPECIALIST_SITE_TYPE_CODE', 'Num_13_full', 'Den_13_full', 'Num_13_exp',
       'Num_26_full', 'Den_26_full', 'Num_26_full_exp', 'Num_26_path',
       'Den_26_path', 'Num_26_path_exp', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_26_wrkast_exp', 'Num_52_full', 'Den_52_full', 'Num_52_full_exp',
       'Num_52_path', 'Den_52_path', 'Num_52_path_exp'],
      dtype='object')

### Aggregating to Site Level 

In [6]:
df_group = df_merged.groupby(['Program', 'CONTRACT_ID', 'SITE_CODE', 'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State', 'SPECIALIST_SITE_TYPE_CODE'])['Num_13_full', 'Den_13_full', 'Num_13_exp', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_full_exp', 'Num_52_path', 'Den_52_path', 'Num_52_path_exp'].apply(lambda x : x.astype(int).sum())

df_group.to_csv('df_group_fin.csv')
df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100




#### Renaming df_group -> df_pre_actual

In [7]:
df_pre_actual = df_group
df_pre_actual.to_csv('C:/Users/AManalo/star_ratings_new/Output Files/test_sites.csv')

In [8]:
def get_actPerf(type, df_group):
    '''Input:
        - type: 'pre' or 'post' quarter
        - df_group: groupBy object that contains site participants with num/den
        Output:
        - returns dataframe with pre/post quarter actual performance for each Contract'''
    
    if type == 'pre':
        df_group['preAct13_full'] = 0 # initialise pre_actual_rate column
        df_group['preAct26_path'] = 0
        df_group['preAct26_full'] = 0
        df_group['preAct26_wrkast'] = 0
        df_group['preAct52_full'] = 0
        df_group['preAct52_path'] = 0
        

    elif type == 'post':
        df_group['postAct13_full'] = 0 # initialise post_actual_rate column
        df_group['postAct26_path'] = 0
        df_group['postAct26_full'] = 0
        df_group['postAct26_wrkast'] = 0
        df_group['postAct52_full'] = 0
        df_group['postAct52_path'] = 0

    

    return df_group

##### Post-Quarter

In [9]:
# reading in the merged file
df_merge_post = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_post.csv')
df_merge_post = df_merge_post.fillna(value = 0)
df_merge_post.head(5)

,Unnamed: 0,Program,JOB_SEEKER_ID,CONTRACT_ID,SITE_CODE,SITE_DESCRIPTION,ESA_CODE,ESA Name,State,SPECIALIST_SITE_TYPE_CODE,Num_13_full,Den_13_full,Num_52,Den_52
0,0,ESS,168470,0212898K,AB31,AimBig Employment AUBURN,4CWS,Northern Sydney,NSW,AALL,0.0,0.0,0.0,0.0
1,1,DMS,451780,0212875D,MO37,AimBig Employment CALOUNDRA,4SUC,QLD North,QLD,MUSK,0.0,0.0,0.0,0.0
2,2,DMS,3606070,0212846K,MM07,AimBig Employment BURWOOD,4INW,South East Sydney,NSW,MENH,1.0,1.0,0.0,0.0
3,3,DMS,618010,0212823C,MN07,AimBig Employment SUNSHINE,4WES,Western Victoria,VIC,MUSK,0.0,0.0,0.0,0.0
4,4,DMS,630040,0212831C,MM67,AimBig Employment ROCKINGHAM,4CWM,Western Australia,WA,MENH,0.0,0.0,0.0,0.0


Calculating Actual Performance

In [10]:
# renaming df_13 ->

df_group = df_merge_post.groupby(['Program', 'CONTRACT_ID', 'SITE_CODE', 'JOB_SEEKER_ID', 'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State', 'SPECIALIST_SITE_TYPE_CODE'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].apply(lambda x : x.astype(int).sum())

df_group.to_csv('df_group_fin.csv')
df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100

df_group = df_group.fillna(0) # if numerator is a 0, then division by a 0 results in a NaN -> but by documentation, should be 0.

df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_test.csv')

KeyError: "Columns not found: 'Num_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_path', 'Num_26_full', 'Den_26_path', 'Num_52_full', 'Den_26_full', 'Den_52_path', 'Den_52_full'"

### Validating Contracts

## Step 2: Expected Performance

Using statistical regression, which takes account of variable Participant and labour market characteristics and the number of Participants in the denominator post-quarterisation, the Star Ratings model calculates the number of outcomes that the Contract could be expected to achieve.

In [ ]:
# Get expected columns from df_merged
df_merged.columns
df_exp = df_merged[['JOB_SEEKER_ID', 'Program', 'CONTRACT_ID', 'SITE_CODE', 'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State', 'SPECIALIST_SITE_TYPE_CODE', 'Num_13_exp', 'Num_26_full_exp', 'Num_52_full_exp', 'Num_52_path_exp']]
       

### Fix later -> since df_26_wrkast and df_26_path EXPECTED are empty -> set to 0


In [ ]:
df_exp['Num_26_wrkast_exp'] = 0
df_exp['Num_26_path_exp'] = 0


Merging df_exp with the denominator from post quarterisation outcomes

In [ ]:
df_merge_post
df_post_den = df_merge_post[["JOB_SEEKER_ID", "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path"]]
df_exp_per = df_exp.merge(df_post_den, on=['JOB_SEEKER_ID'], how='inner')

df_exp_per = df_exp_per.groupby(['Program', 'CONTRACT_ID', 'SITE_CODE', 'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State', 'SPECIALIST_SITE_TYPE_CODE'])[ "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path", 'Num_13_exp', 'Num_26_full_exp', 'Num_52_full_exp', 'Num_52_path_exp', 'Num_26_wrkast_exp', 'Num_26_path_exp'].apply(lambda x : x.astype(int).sum())



Calculating expected performance rate
- for each performance measure

In [ ]:

df_exp_per['exPer_13_full'] = df_exp_per['Den_13_full']/df_exp_per['Num_13_exp'] * 100
df_exp_per['exPer_26_full'] = df_exp_per['Den_26_full']/df_exp_per['Num_26_full_exp'] * 100
df_exp_per['exPer_26_path'] = df_exp_per['Den_26_path']/df_exp_per['Num_26_path_exp'] * 100
df_exp_per['exPer_26_wrkast'] = df_exp_per['Den_26_wrkast']/df_exp_per['Num_26_wrkast_exp'] * 100
df_exp_per['exPer_52_full'] = df_exp_per['Den_52_full']/df_exp_per['Num_52_full_exp'] * 100
df_exp_per['exPer_52_path'] = df_exp_per['Den_52_path']/df_exp_per['Num_52_path_exp'] * 100

df_exp_per = df_exp_per.fillna(0)

df_exp_per

Den_13_full  \
Program CONTRACT_ID SITE_CODE SITE_DESCRIPTION                  ESA_CODE ESA Name             State SPECIALIST_SITE_TYPE_CODE                
DMS     0212789F    FV28      AimBig Employment NORWOOD         4EAA     South Australia      SA    MENH                                 2   
                    MO87      AimBig Employment PROSPECT        4EAA     South Australia      SA    MENH                                10   
        0212790K    FV28      AimBig Employment NORWOOD         4EAA     South Australia      SA    MUSK                                 1   
                    MO87      AimBig Employment PROSPECT        4EAA     South Australia      SA    MUSK                                 2   
        0212791K    FW58      AimBig Employment Pty Ltd. SEATON 4WEA     South Australia      SA    MENH                                 2   
...                                                                                                                                    ...   
ESS     0212903C    FX68      Aimbig Employment RICHMOND        4NEP     South Western Sydney NSW   AALL                                 0   
                    FX78      Aimbig Employment WINDSOR         4NEP     South Western Sydney NSW   AALL                                 0   
        0212904D    A981      AimBig Employment HURSTVILLE      4SGS     South East Sydney    NSW   AALL                                 9   
                    A991      AimBig Employment MIRANDA         4SGS     South East Sydney    NSW   AALL                                 5   
                    NW67      AimBig Employment ROCKDALE        4SGS     South East Sydney    NSW   AALL                                 4   

                                                                                                                               Den_26_full  \
Program CONTRACT_ID SITE_CODE SITE_DESCRIPTION                  ESA_CODE ESA Name             State SPECIALIST_SITE_TYPE_CODE                
DMS     0212789F    FV28      AimBig Employment NORWOOD         4EAA     South Australia      SA    MENH                                 0   
                    MO87      AimBig Employment PROSPECT        4EAA     South Australia      SA    MENH                                 1   
        0212790K    FV28      AimBig Employment NORWOOD         4EAA     South Australia      SA    MUSK                                 0   
                    MO87      AimBig Employment PROSPECT        4EAA     South Australia      SA    MUSK                                 0   
        0212791K    FW58      AimBig Employment Pty Ltd. SEATON 4WEA     South Australia      SA    MENH                                 0   
...                                                                                                                                    ...   
ESS     0212903C    FX68      Aimbig Employment RICHMOND        4NEP     South Western Sydney NSW   AALL                                 0   
                    FX78      Aimbig Employment WINDSOR         4NEP     South Western Sydney NSW   AALL                                 0   
        0212904D    A981      AimBig Employment HURSTVILLE      4SGS     South East Sydney    NSW   AALL                                 0   
                    A991      AimBig Employment MIRANDA         4SGS     South East Sydney    NSW   AALL                                 0   
                    NW67      AimBig Employment ROCKDALE        4SGS     South East Sydney    NSW   AALL                                 0   

                                                                                                                               Den_26_path  \
Program CONTRACT_ID SITE_CODE SITE_DESCRIPTION                  ESA_CODE ESA Name             State SPECIALIST_SITE_TYPE_CODE                
DMS     0212789F    FV28      AimBig Employment NORWOOD         4EAA     South Australia      SA    MENH                                 0   
       

## Step 3: Calculate the Performance Measure Scores (Ratio of Actual Outcomes : Expected Outcomes)

Combined 13, 25, 52 week scores are calculated based on the weightings of each sub type measure. 
Actual rate: actual rate (post-quarter)
Expected rate: expected performance rate


To-Fix: Need to adjust the weights for each performance measure IF the denominator in the pre-quarter outcomes are 0 or very small. 

- ESS: min 20 participants (combined) in the pre-quarterisation denominators for the following: 13-week full outcomes (min 5 participants), ongoing support  
- DMS: min 20 participants in the pre-quarterisation denominator for 13 week full outcomes measure

In [ ]:
# subsetting the post-quarter actual rates for each contract
df_actPer = df_group.reset_index() # converting groupBy obj -> dataframe for subsetting
df_actPer.columns
df_actPer = df_actPer[['Program', 'CONTRACT_ID', 'SITE_CODE', 'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State', 'SPECIALIST_SITE_TYPE_CODE', 'actual_13_full',
       'actual_26_full', 'actual_26_path', 'actual_26_wrkast',
       'actual_52_full', 'actual_52_path']]

# subsetting the expected performance for each contract
df_exp_per = df_exp_per.reset_index()
df_exp_per.columns
df_exp_per = df_exp_per[['Program', 'CONTRACT_ID', 'SITE_CODE', 'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State', 'SPECIALIST_SITE_TYPE_CODE', 'exPer_13_full', 'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast',
       'exPer_52_full', 'exPer_52_path']]

# joining the two dataframes
df_perMea = df_actPer.merge(df_exp_per, on=['Program', 'CONTRACT_ID', 'SITE_CODE', 'SITE_DESCRIPTION', 'ESA_CODE', 'ESA Name', 'State', 'SPECIALIST_SITE_TYPE_CODE'], how='inner')

df_perMea.columns

df_perMea.isna().sum()
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)

Creating the performance measure scores
### replace weightings - DMS, ESS specific 

In [ ]:
df_perMea = df_perMea.groupby(['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path'].apply(lambda x : x.astype(int).sum())

# replace inf with 0 

df_perMea['13_full'] = round(df_perMea['actual_13_full']/df_perMea['exPer_13_full'], ndigits=2)

df_perMea['26_full'] = round(df_perMea['actual_26_full']/df_perMea['exPer_26_full'], ndigits=2)
df_perMea['26_path'] = round(df_perMea['actual_26_path']/df_perMea['exPer_26_path'], ndigits=2)
df_perMea['26_wrkast'] = round(df_perMea['actual_26_wrkast']/df_perMea['exPer_26_wrkast'], ndigits=2)
df_perMea['26_comb'] = round((df_perMea['26_full'] * 0.35) + (df_perMea['26_path'] * 0.05) + (df_perMea['26_wrkast'] * 0.05), ndigits=2) # combined ratio - using DMS weightings 

df_perMea['52_full'] = round(df_perMea['actual_52_full']/df_perMea['exPer_52_full'], ndigits=2)
df_perMea['52_path'] = round(df_perMea['actual_52_path']/df_perMea['exPer_52_path'], ndigits=2)
df_perMea['52_comb'] = round((df_perMea['52_full'] * 0.25) + (df_perMea['52_path'] * 0.05), ndigits=2) # combined ratio - using DMS weightings 

# replace infinity -> 
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)
df_perMea.columns

Index(['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path', '13_full', '26_full', '26_path', '26_wrkast',
       '26_comb', '52_full', '52_path', '52_comb'],
      dtype='object')

### Step 4: Standardise the performance measure scores
Ratio of actual rate pre-standardisation and post-standardisation.

    - scale 0 to 4
    - Using MinMaxScalar

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,4))
# subset the dataframe, then rejoin -> RESET_INDEX first to transform into dataframe, then rejoin, then groupBy




scaler.fit_transform(df_perMea.loc[:, ~df_perMea.columns.isin(['actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path'])])

df_perMea


,,,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,exPer_26_full,exPer_26_path,exPer_26_wrkast,exPer_52_full,exPer_52_path,13_full,26_full,26_path,26_wrkast,26_comb,52_full,52_path,52_comb
CONTRACT_ID,ESA_CODE,SPECIALIST_SITE_TYPE_CODE,,,,,,,,,,,,,,,,,,,,
0212789F,4EAA,MENH,200,100,0,0,0,0,283,0,0,0,0,0,0.71,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0212790K,4EAA,MUSK,200,0,0,0,0,0,150,0,0,0,0,0,1.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0212791K,4WEA,MENH,150,0,0,0,0,0,366,0,0,0,0,0,0.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0212793B,4GOC,MENH,400,0,0,0,0,0,716,0,0,0,0,0,0.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0212795D,4NOB,AALL,546,0,0,0,0,0,772,0,0,0,0,0,0.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0212900K,4INS,AALL,500,0,0,0,0,0,492,0,0,0,0,0,1.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0212901A,4LIV,AALL,100,0,0,0,0,0,62,0,0,0,0,0,1.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0212902B,4MAC,AALL,200,0,0,0,0,0,375,0,0,0,0,0,0.53,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Step 5: Calculate the Overall Performance Score
- Multiplying the standardised performance scores with the DMS/ESS weighting
TO-DO: multiply with the respective DMS/ESS weight

In [ ]:
df_perMea['13_full'] = df_perMea['13_full'] * 0.20
df_perMea['26_full'] = df_perMea['26_full'] * 0.40
df_perMea['52_comb'] = df_perMea['52_comb'] * 0.25

df_perMea['overall_perScore'] = df_perMea['13_full'] + df_perMea['26_full'] + df_perMea['52_comb']

### Step 6: Calculate the Star Rating Percentage
- Calculated using the formula:
str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 


In [ ]:
# bringing in the sub368 file to read in the national averages
df_avg = pd.read_csv("C:/Users/AManalo/star_ratings_new/Sub368_Data_Aug2022.csv")
df_avg.columns

Index(['REPORT_DATE', 'EOM_SEQ', 'EOW_SEQ', 'DES_PROGRAM', 'ORGANISATION_NAME',
       'ORGANISATION_CODE', 'SUMMARY_LINE_TYPE', 'LMR_NAME', 'LMR_CODE',
       'ESA_NAME', 'ESA_CODE', 'CONTRACT_ID', 'SITE_NAME', 'SITE_CODE',
       'LOCALITY', 'SITE_TYPE', 'SPECIALTY_CD', 'START_DATE', 'END_DATE',
       'PROV_EOM_SEQ', 'PROV_EOW_SEQ', 'CONTRACT_START_GROUP',
       'PERFORMANCE_MEASURES', 'RATE', 'NUMERATOR', 'DENOMINATOR',
       'OWN_ORG_ESA_AVG', 'OWN_ORG_LMR_AVG', 'OWN_ORG_NAT_AVG',
       'ALL_ORG_SAME_SPEC_ESA_AVG', 'ALL_ORG_SAME_SPEC_LMR_AVG',
       'ALL_ORG_SAME_SPEC_NAT_AVG', 'ALL_ORG_ESA_AVG', 'ALL_ORG_LMR_AVG',
       'ALL_ORG_NAT_AVG', 'NAT_AVG_ALL_SPEC_JULY_2018'],
      dtype='object')

Joining the sub368 file with the current dataframe to get the national average

str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 

In [ ]:
df_avg = df_avg[['ORGANISATION_NAME',
       'ORGANISATION_CODE', 'ESA_NAME', 'ESA_CODE', 'CONTRACT_ID', 'SITE_NAME', 'SITE_CODE', 'SPECIALTY_CD', 'PERFORMANCE_MEASURES', 'ALL_ORG_NAT_AVG']]

# join df_avg to df_perMea
df_avg = df_avg.rename({df_avg.columns[7] : 'SPECIALIST_SITE_TYPE_CODE'}, axis=1)
df_perMea = df_perMea.merge(df_avg, on=['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'], how='inner')



In [ ]:
# calculating the star rating percentage
df_perMea['star_rating_perc'] = (df_perMea['overall_perScore'] - df_perMea['ALL_ORG_NAT_AVG'])/ (df_perMea['ALL_ORG_NAT_AVG']) * 100